# Guided project: Diving into multi-class classification via logistic regression

This is a **guided project** that we'll step through together as a class to understand multi-class logistic regression!

First, let's import some useful libraries and (optionally) set up some plot aesthetics:

In [ ]:
import pandas as pd
import numpy as np
import os

import seaborn as sns
import cmocean
import matplotlib.pyplot as plt
import pylab
%matplotlib inline
%config InlineBackend.figure_format = 'svg'


viz_style = {
    'font.family': 'sans-serif',
    'font.size':11,
    'axes.titlesize':'large',
    'axes.labelsize':'medium',
    'xtick.labelsize':'small',
    'ytick.labelsize':'small',
    'text.color':'#5B5654',
    'axes.labelcolor':'#5B5654',
    'xtick.color':'#5B5654',
    'ytick.color':'#5B5654',
    'axes.edgecolor':'#5B5654',
    'xtick.top':False,
    'ytick.right':False,
    'axes.spines.top':False,
    'axes.spines.right':False,
    'axes.grid':False,
    'boxplot.showfliers':False,
    'boxplot.patchartist':True
}

plt.style.use(viz_style)

case_dir = '/path/to/materials'

And we're ready to go!

## Read in the data

For this project we'll be using the classic [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/Iris/). This is a very simple dataset, with only 4 features and 3 classes corresponding to different species of Iris: Setosa, Versicolour and Virginica. The features describe the length and width of the petals and sepals.

---

We'll read in the data using `sklearn`'s function `load_iris`. There are a number of other convenience functions that will allow access to other datasets, and if you're interested you can check them out here: https://sklearn.org/datasets/index.html

In [ ]:
# import dataset
from sklearn.datasets import load_iris
data = load_iris()
data.keys() # summarize information in 'data'

Let's take a look at the target and feature names:

In [ ]:
data.target_names

In [ ]:
data.feature_names

Our exploratory data analysis will be easier if we can work with a dataframe, so let's create one now:

In [ ]:
df = pd.DataFrame(data= np.c_[data['data'], data['target']],
                     columns= data['feature_names'] + ['species'])
df

## Exploratory data analysis

Play around with the data and explore each of the 4 features! What are their distributions? How are the three classes distributed? Are any features correlated with the others? What is the relation between each feature (or combination of features) with the target variable? 

---

Recommended visualizations:
* Individual features: [Histogram](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html)
* Feature vs species class: [Box plot](https://seaborn.pydata.org/generated/seaborn.boxplot.html)
* Relations between features and other features/target: [Pair grid](https://seaborn.pydata.org/generated/seaborn.PairGrid.html#seaborn.PairGrid)

What do you think about this dataset? Do you think these features are good predictors of the species of Iris? Why or why not?

## Training and evaluating models

### Defining training and test sets

The first thing we need to do is split our data into a training set and a test set!

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[data.feature_names], df['species'], 
                                                    random_state=0, test_size=0.25, 
                                                    stratify=df['species'])

Notice we're using the `stratify` parameter when we run `train_test_split` here. Check out the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html); what does this parameter do? Why might it be important here? 

### one-vs-rest

Next we'll load in the logistic regression model, the standard scaler for preprocessing, and a couple model evaluation metrics:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

Scale your features, define your model (note we are specifying the one-vs-rest method via the `multi_class='ovr'` parameter setting), and let's see how it does:

In [ ]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

clf = LogisticRegression(multi_class='ovr', random_state=0).fit(X_train_sc, y_train)

In [ ]:
print('--- TRAINING SET ---')
print('Accuracy:', accuracy_score(y_train, clf.predict(X_train_sc)))

print('\n--- TEST SET ---')
print('Accuracy:', accuracy_score(y_test, clf.predict(X_test_sc)))

One way to *visually* assess the performance of a model is by making a **confusion matrix.** For *N* classes, this is an $N\times N$ grid with the true labels on the y-axis and predicted labels on the x-axis. The value of each grid tells you how the predictions for a given class were distributed.

For example, a perfect classifier would have predict class "0" for 100% of the true class "0" samples, and similarly for class "1" and "2." We would see this as a diagonal strip of 100% in the confusion matrix, with 0% everywhere else.

In reality, though, classifiers are rarely perfect. Let's see what ours looks like:

In [ ]:
heatmap_train = sns.heatmap(confusion_matrix(y_train, clf.predict(X_train_sc), normalize="true"), 
                            annot=True, fmt='.2%', cmap="PuBu");
heatmap_train.set_xlabel('Predicted label')
heatmap_train.set_ylabel('True label')
heatmap_train.set_title('Training set');

What are the strengths and weaknesses of the model? Which classes does it have an easier or harder time with?

---

Make a confusion matrix for the **test set**:

What are the differences between the training and test results? Can you explain these differences?

Recall that the one-vs-rest method produces the probabilities of belong to _each_ class for every data point. We can see these via `clf.predict_proba` below:

In [ ]:
clf.predict_proba(X_test_sc)

We might be able to do better than this though! Run a grid search to see which value of the regularization parameter `C` is best for this dataset:

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'C':10**np.linspace(-2,2, 11)} # feel free to change this
gs = GridSearchCV(LogisticRegression(multi_class='ovr', random_state=0), 
                  parameters).fit(X_train_sc, y_train)
print('Best params:', gs.best_params_)

print('\n--- TRAINING SET ---')
print('Accuracy:', accuracy_score(y_train, gs.predict(X_train_sc)))

print('\n--- TEST SET ---')
print('Accuracy:', accuracy_score(y_test, gs.predict(X_test_sc)))

In [ ]:
heatmap_train = sns.heatmap(confusion_matrix(y_train, gs.predict(X_train_sc), normalize="true"), 
                            annot=True, fmt='.2%', cmap="PuBu");
heatmap_train.set_xlabel('Predicted label')
heatmap_train.set_ylabel('True label')
heatmap_train.set_title('Training set');

In [ ]:
heatmap_test = sns.heatmap(confusion_matrix(y_test, gs.predict(X_test_sc), normalize="true"), 
                           annot=True, fmt='.2%', cmap="PuBu")
heatmap_test.set_xlabel('Predicted label')
heatmap_test.set_ylabel('True label')
heatmap_test.set_title('Test set');

What did you find? Should we apply more or less regularization to this problem to improve the accuracy? 

### multinomial

Now let's take a look at the multinomial approach. This will be similar to what you did above, except now we'll set `multi_class='multinomial'` when we define our logistic regression model:

In [ ]:
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

clf = LogisticRegression(multi_class='multinomial', random_state=0).fit(X_train_sc, y_train)

print('--- TRAINING SET ---')
print('Accuracy:', accuracy_score(y_train, clf.predict(X_train_sc)))

print('\n--- TEST SET ---')
print('Accuracy:', accuracy_score(y_test, clf.predict(X_test_sc)))

And let's check out the confusion matrices for the training and test sets as well:

In [ ]:
heatmap_train = # fill out

In [ ]:
heatmap_test = # fill out

How does this compare to the OVR method?

Let's run a grid search here too:

In [ ]:
parameters = {'C':10**np.linspace(-2,2, 11)} # feel free to change this
gs = GridSearchCV(LogisticRegression(multi_class='multinomial', random_state=0), 
                  parameters).fit(X_train_sc, y_train)
print('Best params:', gs.best_params_)

print('\n--- TRAINING SET ---')
print('Accuracy:', accuracy_score(y_train, gs.predict(X_train_sc)))

print('\n--- TEST SET ---')
print('Accuracy:', accuracy_score(y_test, gs.predict(X_test_sc)))

In [ ]:
heatmap_train = # fill out

In [ ]:
heatmap_test = # fill out